# CSE 258 Assignment 1 Read
**Ming Ki Toby Cheng**

In [1]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
from sklearn import linear_model
import numpy
import random

import matplotlib.pyplot as plt

In [2]:
def readGz(path):
    for l in gzip.open(path, "rt"):
        yield eval(l)


def readCSV(path):
    f = gzip.open(path, "rt")
    f.readline()
    for l in f:
        yield l.strip().split(",")

In [3]:
users = []
books = []
ratings = []

In [4]:
for user, book, _ in readCSV("train_Interactions.csv.gz"):
    users.append(user)
    books.append(book)
    ratings.append(_)

In [5]:
all_data = list(zip(users, books, ratings))

In [6]:
userCount = defaultdict(set)
allBooks = set()

for user, book, _ in readCSV("train_Interactions.csv.gz"):
    userCount[user].add(book)
    allBooks.add(book)

In [8]:
# Defining sets for users and books
usersPerBook = defaultdict(set)
booksPerUser = defaultdict(set)
for user, book,_ in all_data:
    usersPerBook[book].add(user)
    booksPerUser[user].add(book)

In [10]:
bookCount = defaultdict(int)
totalRead = 0

for user, book,_ in all_data:
    bookCount[book] += 1
    totalRead += 1

In [11]:
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

In [12]:
# Defining Jaccard similarity function
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return (numer / denom)

In [31]:
def predictionJaccard(u, b):
    similarities = []
    books = booksPerUser[u]
    users = usersPerBook[b]
    for book in books:
        users_other = usersPerBook[book]
        sim = Jaccard(users, users_other)
        similarities.append((sim))
    similarities.sort(reverse = True)
    return(numpy.mean(similarities))

In [14]:
# Defining Cosine similarity function
def Cosine(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1) * len(s2)
    if denom == 0:
        return(0)
    else:
        return(numer / denom)

In [15]:
# Defining separate Cosine Prediction function
def predictionCosine(u, b):
    cos_sim = []
    books = booksPerUser[u]
    users = usersPerBook[b]
    for book in books:
        users_other = usersPerBook[book]
        csim = Cosine(users, users_other)
        cos_sim.append((csim, book))
        
    cos_sim.sort(reverse = True)

    return(cos_sim[0][0])

In [34]:
# Defining sets for users and books
usersPerBook_test = defaultdict(set)
booksPerUser_test = defaultdict(set)
readPerPair_test = defaultdict(int)
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        # header
        continue
    u, b = l.strip().split("-")
    usersPerBook_test[b].add(u)
    booksPerUser_test[u].add(b)
    
jaccard_user_test = defaultdict(dict)
for i in booksPerUser_test:
    books = booksPerUser_test[i]
    for books in booksPerUser_test[i]:
        jaccard_user_test[i][books] = (predictionJaccard(i, books)*bookCount[books])   

In [35]:
for i in jaccard_user_test:
    jaccard_user_test[i] = sorted(jaccard_user_test[i].items(), key=lambda x:x[1])

In [36]:
readPerPair_test = defaultdict(int)
for i in jaccard_user_test:
    num = len(jaccard_user_test[i])
    unread = jaccard_user_test[i][:num//2]
    read = jaccard_user_test[i][num//2:]
    for pair in read:
        readPerPair_test[i,pair[0]] = 1
    for pair2 in unread:
        readPerPair_test[i,pair2[0]] = 0

In [37]:
# Writing predictions of test set to file
predictions = open("predictions_Read_Assignment1_v3.1.txt", "w")
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        # header
        predictions.write(l)
        continue
    u, b = l.strip().split("-")
    if readPerPair_test[u,b] == 1:
        predictions.write(u + "-" + b + ",1\n")
    else:
        predictions.write(u + "-" + b + ",0\n")

predictions.close()

**Kaggle Username: tobycheng or Toby Cheng**

**Kaggle Read Accuracy: 0.74050**